In [4]:
# modified code slightly
# original code https://github.com/microsoft/CameraTraps/blob/main/data_management/lila/download_lila_subset.py
#
# download_lila_subset.py
#
# Example of how to download a list of files from LILA, e.g. all the files
# in a data set corresponding to a particular species.
#

#%% Constants and imports

import json
import urllib.request
import tempfile
import zipfile
import os
import shutil
import time

from tqdm import tqdm
from multiprocessing.pool import ThreadPool
from urllib.parse import urlparse


def download_image(species, datasets):
    """
    variables:
        species = string
        datasets = list of existing lila datasets
    """
    # LILA camera trap master metadata file
    metadata_url = 'http://lila.science/wp-content/uploads/2020/03/lila_sas_urls.txt'

    # In this example, we're using the Missouri Camera Traps data set and the Caltech Camera Traps dataset
    datasets_of_interest = datasets

    # All lower-case; we'll convert category names to lower-case when comparing
    species_of_interest = [species]

    # We'll write images, metadata downloads, and temporary files here
    lila_local_base = 'data/'

    output_dir = os.path.join(lila_local_base, 'camera_trap')
    os.makedirs(output_dir, exist_ok=True)

    metadata_dir = os.path.join(lila_local_base, 'metadata_camera_trap')
    os.makedirs(metadata_dir, exist_ok=True)

    # We will demonstrate two approaches to downloading, one that loops over files
    # and downloads directly in Python, another that uses AzCopy.

    overwrite_files = False

    # Number of concurrent download threads (when not using AzCopy) (AzCopy does its
    # own magical parallelism)
    n_download_threads = 50

    max_images_per_dataset = 100


    #%% Support functions

    def download_url(url, destination_filename=None, force_download=False, verbose=True):
        """
        Download a URL (defaulting to a temporary file)
        """

        if destination_filename is None:
            temp_dir = os.path.join(tempfile.gettempdir(),'lila')
            os.makedirs(temp_dir,exist_ok=True)
            url_as_filename = url.replace('://', '_').replace('.', '_').replace('/', '_')
            destination_filename = \
                os.path.join(temp_dir,url_as_filename)

        if (not force_download) and (os.path.isfile(destination_filename)):
            print('Bypassing download of already-downloaded file {}'.format(os.path.basename(url)))
            return destination_filename

        if verbose:
            print('Downloading file {} to {}'.format(os.path.basename(url),destination_filename),end='')

        os.makedirs(os.path.dirname(destination_filename),exist_ok=True)
        urllib.request.urlretrieve(url, destination_filename)  
        assert(os.path.isfile(destination_filename))

        if verbose:
            nBytes = os.path.getsize(destination_filename)    
            print('...done, {} bytes.'.format(nBytes))
        return destination_filename


    def download_relative_filename(url, output_base, verbose=False):
        """
        Download a URL to output_base, preserving relative path
        """

        p = urlparse(url)
        # remove the leading '/'
        assert p.path.startswith('/'); relative_filename = p.path[1:]
        destination_filename = os.path.join(output_base,relative_filename)
        download_url(url, destination_filename, verbose=verbose)
        

    def unzip_file(input_file, output_folder=None):
        """
        Unzip a zipfile to the specified output folder, defaulting to the same location as
        the input file    
        """

        if output_folder is None:
            output_folder = os.path.dirname(input_file)

        with zipfile.ZipFile(input_file, 'r') as zf:
            zf.extractall(output_folder)


    #%% Download and parse the metadata file

    # Put the master metadata file in the same folder where we're putting images
    p = urlparse(metadata_url)
    metadata_filename = os.path.join(metadata_dir,os.path.basename(p.path))
    download_url(metadata_url, metadata_filename)

    # Read lines from the master metadata file
    with open(metadata_filename,'r') as f:
        metadata_lines = f.readlines()
    metadata_lines = [s.strip() for s in metadata_lines]

    # Parse those lines into a table
    metadata_table = {}

    for s in metadata_lines:

        if len(s) == 0 or s[0] == '#':
            continue

        # Each line in this file is name/base_url/json_url/[box_url]
        tokens = s.split(',')
        assert len(tokens)==4
        url_mapping = {'sas_url':tokens[1],'json_url':tokens[2]}
        metadata_table[tokens[0]] = url_mapping

        assert 'https' not in tokens[0]
        assert 'https' in url_mapping['sas_url']
        assert 'https' in url_mapping['json_url']


    #%% Download and extract metadata for the datasets we're interested in

    for ds_name in datasets_of_interest:

        assert ds_name in metadata_table
        json_url = metadata_table[ds_name]['json_url']

        p = urlparse(json_url)
        json_filename = os.path.join(metadata_dir,os.path.basename(p.path))
        download_url(json_url, json_filename)

        # Unzip if necessary
        if json_filename.endswith('.zip'):

            with zipfile.ZipFile(json_filename,'r') as z:
                files = z.namelist()
            assert len(files) == 1
            unzipped_json_filename = os.path.join(metadata_dir,files[0])
            if not os.path.isfile(unzipped_json_filename):
                unzip_file(json_filename,metadata_dir)        
            else:
                print('{} already unzipped'.format(unzipped_json_filename))
            json_filename = unzipped_json_filename

        metadata_table[ds_name]['json_filename'] = json_filename

    # ...for each dataset of interest


    #%% List of files we're going to download (for all data sets)

    # Flat list or URLS, for use with direct Python downloads
    urls_to_download = []

    # For use with azcopy
    downloads_by_dataset = {}

    for ds_name in datasets_of_interest:

        json_filename = metadata_table[ds_name]['json_filename']
        sas_url = metadata_table[ds_name]['sas_url']

        # This may or may not be a SAS URL
        if '?' in sas_url:
            base_url = sas_url.split('?')[0]        
            sas_token = sas_url.split('?')[1]
            assert not sas_token.startswith('?')
        else:
            sas_token = ''
            base_url = sas_url

        assert not base_url.endswith('/')


        ## Open the metadata file

        with open(json_filename, 'r') as f:
            data = json.load(f)

        categories = data['categories']
        for c in categories:
            c['name'] = c['name'].lower()
        category_id_to_name = {c['id']:c['name'] for c in categories}
        annotations = data['annotations']
        images = data['images']


        ## Build a list of image files (relative path names) that match the target species

        category_ids = []

        for species_name in species_of_interest:
            matching_categories = list(filter(lambda x: x['name'] == species_name, categories))
            if len(matching_categories) == 0:
                continue
            assert len(matching_categories) == 1
            category = matching_categories[0]
            category_id = category['id']
            category_ids.append(category_id)

        print('Found {} matching categories for data set {}:'.format(len(category_ids),ds_name))

        if len(category_ids) == 0:
            continue

        for i_category,category_id in enumerate(category_ids):
            print(category_id_to_name[category_id],end='')
            if i_category != len(category_ids) -1:
                print(',',end='')
        print('')

        # Retrieve all the images that match that category
        image_ids_of_interest = set([ann['image_id'] for ann in annotations if ann['category_id'] in category_ids])

        print('Selected {} of {} images for dataset {}'.format(len(image_ids_of_interest),len(images),ds_name))

        # Retrieve image file names
        filenames = [im['file_name'] for im in images if im['id'] in image_ids_of_interest]
        assert len(filenames) == len(image_ids_of_interest)

        if max_images_per_dataset is not None and len(filenames) > max_images_per_dataset:
            print('Taking the first {} of {} images for {}'.format(
                max_images_per_dataset,len(filenames),ds_name))
            filenames = filenames[0:max_images_per_dataset]

        # Convert to URLs
        for fn in filenames:        
            url = base_url + '/' + fn
            urls_to_download.append(url)

        downloads_by_dataset[ds_name] = {'sas_url':sas_url,'filenames':filenames}

    # ...for each dataset

    print('Found {} images to download'.format(len(urls_to_download)))


    #%% Download those image files

    # Loop over files
    print('Downloading images for {0} without azcopy'.format(species_of_interest))

    if n_download_threads <= 1:
        for url in tqdm(urls_to_download):        
            download_relative_filename(url,output_dir,verbose=True)

    else:
        pool = ThreadPool(n_download_threads)        
        tqdm(pool.imap(lambda s: download_relative_filename(s,output_dir,verbose=False), urls_to_download), total=len(urls_to_download))
    
    
    # karoline addition
    # move images to species specific folder
    print("moving images")
    print(len(urls_to_download))
    
    counter = 0
    
    for image in urls_to_download:
        org_dir = "data/camera_trap/"
 
        filename = image.rsplit('/', 1)[-1]
        destination_folder = org_dir + species
        destination = os.path.join(destination_folder, filename)

        if not os.path.exists(destination_folder):
            print("creating animal directory")
            os.mkdir(destination_folder)

        print(os.path.exists("data/camera_trap/" + image[42:]))
        print("data/camera_trap/" + image[42:])
        # make sure the image is downloaded properly before we move it
        count_loop = 0
        while not os.path.exists("data/camera_trap/" + image[42:]):
            time.sleep(2)
            count_loop += 1
            if count_loop > 10:
                break
        if os.path.exists("data/camera_trap/" + image[42:]):
            shutil.move("data/camera_trap/" + image[42:], destination)
        
        counter += 1
    
    print(f'{counter} images done!')

### Available datasets from lila
- Caltech Camera Traps
- ENA24
- Missouri Camera Traps
- NACTI
- WCS Camera Traps
- Wellington Camera Traps
- Island Conservation Camera Traps
- Channel Islands Camera Traps
- Idaho Camera Traps
- Snapshot Serengeti
- Snapshot Karoo
- Snapshot Kgalagadi
- Snapshot Enonkishu
- Snapshot Camdeboo
- Snapshot Mountain Zebra
- Snapshot Kruger
- SWG Camera Traps
- Orinoquia Camera Traps

In [5]:
animals = ["hippopotamus", "rhinoceros", "impala", "lionfemale", "lionmale", "lioncub", "giraffe", 
           "baboon", "warthog", "hyenaspotted", "hyenastriped", "hyenabrown", "eland", "waterbuck", 
           "wildebeest", "leopard", "cheetah", "wilddog", "zebra", "zebraburchells", "zebramountain", 
           "buffalo", "elephant", "monkeyvervet", "pangolin", "secretarybird"]
datasets = ["Snapshot Kruger", 
            "Snapshot Enonkishu", 
            "Snapshot Serengeti", 
            "Snapshot Karoo", 
            "Snapshot Kgalagadi",
            "Snapshot Mountain Zebra",
            "Snapshot Camdeboo"]

In [6]:
%%time

for i in animals:
    download_image(i, datasets)
    print(f"{i} finished------------------------------------------------")

Bypassing download of already-downloaded file lila_sas_urls.txt
Bypassing download of already-downloaded file SnapshotKruger_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotKruger_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotEnonkishu_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotEnonkishu_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotSerengeti_S1-11_v2_1.json.zip
data/metadata_camera_trap/SnapshotSerengeti_S1-11_v2.1.json already unzipped
Bypassing download of already-downloaded file SnapshotKaroo_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotKaroo_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotKgalagadi_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotKgalagai_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotMountainZebra_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotMountainZebra_S1_v1.0.json already unzipped
Bypassi

  0%|                                                   | 0/205 [00:00<?, ?it/s]


moving images
205
creating animal directory
False
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0019.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0021.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0023.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0027.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0056.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0071.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0079.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0129.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0130.JPG
True
data/camera_trap/snapshot-safari/KRU/KRU_public/KRU_S1/11/11_R1/KRU_S1_11_R1_IMAG0131.JPG
True


True
data/camera_trap/snapshot-safari/ENO/ENO_public/ENO_S1/D02/D02_R1/ENO_S1_D02_R1_IMAG0138.JPG
True
data/camera_trap/snapshot-safari/ENO/ENO_public/ENO_S1/D02/D02_R1/ENO_S1_D02_R1_IMAG0139.JPG
True
data/camera_trap/snapshot-safari/ENO/ENO_public/ENO_S1/D02/D02_R1/ENO_S1_D02_R1_IMAG0306.JPG
True
data/camera_trap/snapshot-safari/ENO/ENO_public/ENO_S1/D03/D03_R1/ENO_S1_D03_R1_IMAG2784.JPG
True
data/camera_trap/snapshot-safari/ENO/ENO_public/ENO_S1/D04/D04_R2/ENO_S1_D04_R2_IMAG0025.JPG
True
data/camera_trap/snapshot-safari/ENO/ENO_public/ENO_S1/D04/D04_R2/ENO_S1_D04_R2_IMAG0105.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/D02/D02_R1/S1_D02_R1_PICT1349.JPG
False
data/camera_trap/snapshotserengeti-unzipped/S1/D02/D02_R1/S1_D02_R1_PICT1350.JPG
False
data/camera_trap/snapshotserengeti-unzipped/S1/D02/D02_R1/S1_D02_R1_PICT1351.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/D03/D03_R1/S1_D03_R1_PICT0043.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/D03/D03_R1/S1_

205 images done!
hippopotamus finished------------------------------------------------
Bypassing download of already-downloaded file lila_sas_urls.txt
Bypassing download of already-downloaded file SnapshotKruger_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotKruger_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotEnonkishu_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotEnonkishu_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotSerengeti_S1-11_v2_1.json.zip
data/metadata_camera_trap/SnapshotSerengeti_S1-11_v2.1.json already unzipped
Bypassing download of already-downloaded file SnapshotKaroo_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotKaroo_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotKgalagadi_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotKgalagai_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotMountainZebra_S1_v1.0.json.zi

...

True
data/camera_trap/snapshotserengeti-unzipped/SER_S11/P06/P06_R1/SER_S11_P06_R1_IMAG0597.JPG
True
data/camera_trap/snapshotserengeti-unzipped/SER_S11/P06/P06_R1/SER_S11_P06_R1_IMAG0598.JPG
True
data/camera_trap/snapshotserengeti-unzipped/SER_S11/P06/P06_R1/SER_S11_P06_R1_IMAG0599.JPG
True
data/camera_trap/snapshotserengeti-unzipped/SER_S11/P06/P06_R1/SER_S11_P06_R1_IMAG0783.JPG
True
data/camera_trap/snapshotserengeti-unzipped/SER_S11/P06/P06_R1/SER_S11_P06_R1_IMAG0784.JPG
True
data/camera_trap/snapshotserengeti-unzipped/SER_S11/P06/P06_R1/SER_S11_P06_R1_IMAG0785.JPG
7 images done!
pangolin finished------------------------------------------------
Bypassing download of already-downloaded file lila_sas_urls.txt
Bypassing download of already-downloaded file SnapshotKruger_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotKruger_S1_v1.0.json already unzipped
Bypassing download of already-downloaded file SnapshotEnonkishu_S1_v1.0.json.zip
data/metadata_camera_trap/SnapshotEnonkishu_S1_v1

  0%|                                                   | 0/134 [00:00<?, ?it/s]

moving images
134
creating animal directory


False
data/camera_trap/snapshotserengeti-unzipped/S1/D07/D07_R3/S1_D07_R3_PICT0359.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/D07/D07_R3/S1_D07_R3_PICT0360.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/D07/D07_R3/S1_D07_R3_PICT0361.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E05/E05_R3/S1_E05_R3_PICT0110.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E05/E05_R3/S1_E05_R3_PICT0111.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E05/E05_R3/S1_E05_R3_PICT0112.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E05/E05_R3/S1_E05_R3_PICT0194.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E05/E05_R3/S1_E05_R3_PICT0195.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E05/E05_R3/S1_E05_R3_PICT0196.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E05/E05_R4/S1_E05_R4_PICT0017.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E05/E05_R4/S1_E05_R4_PICT0018.JPG
True
data/camera_trap/snapshotserengeti-unzipped/S1/E

True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/A10/A10_R1/KGA_S1_A10_R1_IMAG0022.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/A10/A10_R1/KGA_S1_A10_R1_IMAG0023.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/A10/A10_R1/KGA_S1_A10_R1_IMAG0024.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0146.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0147.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0148.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0149.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0150.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0151.JPG
True
data/camera_trap/snapshot-safari/KGA/KGA_public/KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0380.JPG
True
data/camera_tra